# V4 Neural Data Challenge - Model Fits

$\hspace{3mm}$To begin, we extracted several basic features which we thought could explain neural responses to some degree. We started by using the raw RGB values, the pixel values in LAB space (an alternate color space), the power spectrum of the 2D fourier transform, the components of a discrete wavelet transformation of the image (using a Gabor filter), and finally several basic image statistics. These statistics are calculated only on the center of the image. They are: the minimum pixel value, the maximum pixel value, the mean pixel value, and the standard deviation of the pixel values. These were computed for each channel (RGB). <br><br>
$\hspace{3mm}$ We also included the output of Alexnet as features to regress upon. Briefly, we normalized and rescaled the images, ran them through Alexnet, and got the response of all 5 convolutional layers. We only used the responses to the center of the image (e.g., conv1 is 55 x 55 x 196, we only use the middle 20 x 20 x 196 block of responses). 

The code to compute the basic image features can be found at: <br>
https://github.com/lambdaloop/v4-challenge/blob/master/compute_features.py

The code to compute the Alexnet features can be found at: <br>
https://github.com/lambdaloop/v4-challenge/blob/master/nn_output_aspredictor.py

In [15]:
import pandas as pd
import numpy as np
from scipy import stats, signal
from tqdm import trange, tqdm
from matplotlib.pyplot import *

%matplotlib inline

In [16]:
# A bunch of machine learning stuff
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics.scorer import make_scorer
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.decomposition import PCA

import umap

In [17]:
# figure styling
import seaborn as sns
sns.set(style='ticks', palette='colorblind')

rcParams['figure.figsize'] = (13,4.5)
rcParams['figure.dpi'] = 200

In [18]:
images = np.load('../data/stim.npy')

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/sub.csv')

responses = np.array(train.iloc[:, 1:])

dd = np.load('../data/features.npz')
features = dict()
for k in dd.keys():
    features[k] = dd[k]

    
conv_train = np.load('../data/conv_train.npy').flat[0]
conv_test = np.load('../data/conv_test.npy').flat[0]

conv_features = dict()
for k in conv_train.keys():
    conv_features[k] = np.vstack([conv_test[k], conv_train[k]])

In [19]:
features.keys()

dict_keys(['raw', 'LAB', 'edges', 'fourier', 'gabor', 'stats', 'stats_LAB', 'stats_HSV', 'stats_edges'])

In [20]:
conv_features.keys()

dict_keys(['conv1', 'conv2', 'conv3', 'conv4', 'conv5'])

## Preprocessing and choosing features

In the following section, we preprocess all the features in order to feed them our regressor.

We found that any fitting algorithm performed poorly with too many predictors, so we projected each feature with too many predictors onto its principal components. 

In addition, we also suspected that the visual system may be responding to the position of images on a manifold. In order to get at this manifold, we also computed a projection using the [UMAP](https://github.com/lmcinnes/umap) embedding. 

The number of components for both PCA and UMAP was optimized manually to minimize the cross-validated mean rmse. It is likely that optimizing these parameters with a proper search could lead to a better solution.


In [40]:
pcs_lab = PCA(n_components=50).fit_transform(features['LAB'])
pcs_fourier = PCA(n_components=30).fit_transform(features['fourier'])
pcs_gabor = PCA(n_components=20).fit_transform(features['gabor'])

In [41]:
pcs_raw = PCA(n_components=20).fit_transform(features['raw'])
pcs_edges = PCA(n_components=20).fit_transform(features['edges'])

In [37]:
pcs_conv1 = PCA(n_components=30).fit_transform(conv_features['conv1'])
pcs_conv2 = PCA(n_components=20).fit_transform(conv_features['conv2'])
pcs_conv3 = PCA(n_components=20).fit_transform(conv_features['conv3'])
pcs_conv4 = PCA(n_components=20).fit_transform(conv_features['conv4'])
pcs_conv5 = PCA(n_components=20).fit_transform(conv_features['conv5'])

In [24]:
embedding = umap.UMAP(metric='correlation', min_dist=1.0, n_components=100)
X_embed = embedding.fit_transform(features['raw'])

In [25]:
embedding = umap.UMAP(metric='correlation', min_dist=1.0, n_components=10)
X_embed_edges = embedding.fit_transform(features['edges'])

embedding = umap.UMAP(metric='correlation', min_dist=1.0, n_components=10)
X_embed_fourier = embedding.fit_transform(features['fourier'])

embedding = umap.UMAP(metric='correlation', min_dist=1.0, n_components=10)
X_embed_lab = embedding.fit_transform(features['LAB'])

In [42]:
X_all = np.hstack([
   X_embed,  
   X_embed_edges,
   X_embed_lab, 
   X_embed_fourier,
   features['stats'], 
   features['stats_LAB'], 
   features['stats_edges'],
   pcs_lab,
   pcs_fourier, 
   pcs_gabor,
   pcs_edges,
   pcs_raw,
   pcs_conv1, 
   pcs_conv2, 
   pcs_conv3,
   pcs_conv4,  
   pcs_conv5,
                  ])

X_all.shape

(601, 470)

## Fitting and evaluating the model

Next, we take all the features and fit the model neuron by neuron. 
We played around with a couple models in scikit-learn, and found the extra-trees regressor to perform the best. This estimator averages a bunch of decision trees fit on the sub-samples of the data. This allows the model to fit nonlinearities and interactions but controlling for overfitting. 

Here, we fit and evaluate an extra-trees regressor with some simpler parameters to check our cross-validation score quickly. We print out the mean rmse with and without neuron 17. 

In [43]:
model = ExtraTreesRegressor(max_depth=None, max_features='sqrt', criterion="mse", n_estimators=100, n_jobs=1)

all_scores = []
for i in range(responses.shape[1]):
    vals = responses[:, i]
    good = ~np.isnan(vals)
    scores = cross_val_score(model, X_all[50:][good], vals[good], 
                             scoring=make_scorer(mean_squared_error), cv=3)
    scores = np.sqrt(scores)
    print(i, np.mean(scores))
    all_scores.append(np.mean(scores))
    
print('mean rmse:', np.mean(all_scores))
print('mean rmse without 17:', np.mean(all_scores[:-1]))

0 0.9401612957630908
1 0.6672550582675812
2 0.6425172425653005
3 0.26917291291149953
4 0.7629908346594553
5 1.039534896806068
6 1.0740585615427993
7 0.4782933509971528
8 0.5247241922388973
9 0.5402333861880092
10 0.29454477061921747
11 0.8026178731737081
12 0.6909614742141151
13 0.5660701433568197
14 0.7181016896970075
15 0.5727938179694713
16 1.1006788621546244
17 0.4338199426080432
mean rmse: 0.6732516836518256
mean rmse without 17: 0.6873359037132246


## Obtaining predictions

Finally, we use the same model but with more estimators as the basis for our final prediction. We generate two sets of predictions. The first consists of the predictions of extra-trees model fit on each neuron. The second has all the same predictions, but with a prediction of the mean for neuron 17. 

From our tests, we found that for any model or feature set, we always obtained a negative R^2 value for neuron 17 predictions, which means it is better to predict the mean response than whatever our model outputs. We suspect that neuron 17 does not respond to any visual features consistently. 

In [44]:
model = ExtraTreesRegressor(max_depth=None, n_estimators=400, n_jobs=4)

new_test = test.copy()

for i in trange(responses.shape[1]):
    vals = responses[:, i]
    good = ~np.isnan(vals)
    model.fit(X_all[50:][good], vals[good])
    pred = model.predict(X_all[:50])
    new_test.iloc[:, i+1] = pred

100%|██████████| 18/18 [01:17<00:00,  4.32s/it]


In [45]:
new_test.to_csv('../data/output_umap_nnet.csv', index=False)

In [46]:
new_test_17 = new_test.copy()
new_test_17.iloc[:, 18] = np.mean(responses[:, 17])
new_test_17.to_csv('../data/output_umap_nnet_blank17.csv', index=False)

## Things we tried but didn't pan out

Finally, in the interest of science, we report on the things we tried but couldn't get to work as well as our current solution

- Using features from VGG19 instead of AlexNet
- Predicting on features directly instead of PCs of features (too slow)
- Using ridge or lasso regression over any set of features (including VGG or AlexNet)
- Using random forest model instead of extra-trees model (slower and worse)